In [1]:
import pandas as pd
import numpy as np

data_file_name = 'data.csv'
labels_file_name = 'what_answer_codes_mean.csv'
questions_file_name = 'what_questsions_are.csv'

data = pd.read_csv(data_file_name)
labels = pd.read_csv(labels_file_name)
questions = pd.read_csv(questions_file_name)

In [3]:
# remove data where country is Nan
data = data[data['Residency'] != 'Nan']
data = data.dropna()
data = data[data['var'] == data['var'].astype(str)]

# remove data in what_questions_are where "question" is not a string
questions = questions[questions['question'] == questions['question'].astype(str)]
# get and remove the question data where [var] is NA
group_header_statements = questions[questions['row_id'] == 0]
questions = questions[questions['row_id'] != 0]

# get unique countries
countries = data['Residency'].unique()
# get unique questions
unique_questions = data['var'].unique()

In [17]:
country_dict = {}

for index, row in data.iterrows():
    country_dict[row['Residency']] = country_dict.get(row['Residency'], {})
    country_dict[row['Residency']][row['var']] = country_dict[row['Residency']].get(row['var'], [])
    country_dict[row['Residency']]['DemGen'] = country_dict[row['Residency']].get('DemGen', [])
    country_dict[row['Residency']]['quota_age'] = country_dict[row['Residency']].get('quota_age', [])
    country_dict[row['Residency']][row['var']].append(row['code'])
    country_dict[row['Residency']]['DemGen'].append(row['DemGen'])
    country_dict[row['Residency']]['quota_age'].append(row['quota_age'])
    country_dict[row['Residency']]['EndDate'] = country_dict[row['Residency']].get('EndDate', [])
    country_dict[row['Residency']]['EndDate'].append(row['EndDate'][0:10] + ' 00:00:00')

In [22]:
for key in country_dict.keys():
    for key2 in country_dict[key].keys():
        if key2 == 'EndDate':
            dates = country_dict[key][key2]
            mean = (np.array(dates, dtype='datetime64[s]')
            .view('i8')
            .mean()
            .astype('datetime64[s]'))
            country_dict[key][key2] = mean

In [ ]:
df = pd.DataFrame.from_dict(country_dict, orient='index')
df.to_json('data_with_dates.json')

In [ ]:
df = pd.read_json('data.json')
# print(df)
# print column names
print(df['GenSocTrust'])

In [29]:
for key in country_dict.keys():
    for key2 in country_dict[key].keys():
        if key2 == 'EndDate':
            print(key, key2, country_dict[key][key2])

KeyError: "['EndDate'] not found in axis"